# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening)
	* [Initial Code](#Initial-Code)
	* [Add moves counter](#Add-moves-counter)
	* [negamaxIDS](#negamaxIDS)
	* [negamaxIDSab](#negamaxIDSab)
	* [Grading](#Grading)
	* [Extra Credit](#Extra-Credit)


## Initial Code 

The below given code to compute Effective Branch Factor (EBF) was developed and implemented as a part of Assignment 2  

In [1]:
nodes=0
netdepth=0

def ebf(nNodes, depth, precision=0.01):
    if depth==0:
        if nNodes==1:
            return 1.0
        else:
            return 0.0
    global nodes
    nodes=nNodes
    global netdepth
    netdepth=depth
    precision=precision
    x0=nNodes**(1/(depth+1))
    result = solve(quadratic, x0, precision) 
    return result

def derivative(f, x, h):
      return (f(x+h) - f(x-h)) / (2.0*h)  

def quadratic(x):
    return ((1-x**(netdepth +1))/(1-x))-nodes     

def solve(f, x0, h):
    lastX = x0
    nextX = lastX + 10* h  
    while (abs(lastX - nextX) > h):  
        newY = f(nextX)                        
        lastX = nextX
        nextX = lastX - newY / derivative(f, lastX, h)  
    return nextX        

def movesExplored():
    return game.movesExplored

The function to compute Negamax was provided by Prof. Chuck Anderson

In [2]:
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

This section of the code computes Negamax using Iterative Deeping Search

In [3]:
def negamaxIDS(game, depthLeft):
    for depth in range (depthLeft):
        score,move = negamax(game,depth)
        if score is None:
            continue
        a = game.getWinningValue()
        if score == a:
            return score, move
    return score,move

This section of the code computes Negamax using Iterative Deeping Search, with alpha, beta Pruning

In [4]:
def negamaxab(game, depthLeft):
    inf = float ('inf')
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        inf = -inf
        # Apply a move to current state
        game.makeMove(move)
        value, _ = negamaxab(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        inf = max(bestValue, inf)
        if bestValue >= inf:
            return bestValue,bestMove
    return bestValue, bestMove

def negamaxIDSab(game, depthLeft):
    for depth in range (depthLeft):
        score,move = negamaxab(game,depth)
        if score is None:
            continue
        a = game.getWinningValue()
        if score == a:
            return score, move
    return score,move

In [5]:
class TTT(object):
    def __init__(self):
        self.board = [' ']*9
        self.player = 'X'
        if False:
            self.board = ['X', 'X', ' ', 'X', 'O', 'O', ' ', ' ', ' ']
            self.player = 'O'
        self.playerLookAHead = self.player
        self.movesExplored = 0 

    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            self.Winner='X'
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            self.Winner='O'
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            self.Winner='Tie'
            return 0
        else:
            return None  ########################################################## CHANGED FROM -0.1

    def isOver(self):
        return self.getUtility() is not None

    def makeMove(self, move):
        self.movesExplored = self.movesExplored + 1 
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s
    
    def getWinningValue(self):
        return 1

In [6]:
def opponent(board):
    return board.index(' ')

def playGame(game,opponent,depthLimit):
    total= -1
    turns= 0
    print(game)
    while not game.isOver():
        if game.player == 'X':
            turns = turns + 1
            total = total + 2
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
            
    print ('Negamax made {} moves. {} moves explored for ebf({}, {}) of {}'.format(turns,game.movesExplored,game.movesExplored,total,ebf(game.movesExplored,total)))
    return 

In [7]:
game = TTT()
playGame(game,opponent,20)

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 
Negamax made 4 moves. 558334 moves explored for ebf(558334, 7) of 6.464860834165034


## playGames

Now duplicate the game playing loop so three complete tic-tac-toe games are played.  Call this new version `playGames`. For the first game, use `negamax`. For the second game, use `negamaxIDS`.  For the third game, use `negamaxIDSab`.  At the end of each game, print the number of X's in the final board, the number moves explored, the depth of the game which is the number of moves made by X and O, and the effective branching factor.  When you run `playGames` you should see the tic-tac-toe positions after each move and, after all games are done, a line for each game like the following lines, which were

    negamax made 4 moves. 558334 moves explored for ebf(558334, 7) of 6.46
    negamaxIDS made 3 moves. 23338 moves explored for ebf(23338, 5) of 7.26
    negamaxIDSab made 3 moves 6053 moves explored for ebf(6053, 5) of 5.48

Your results may be different. 

The value of the depth is the total number of moves made by X and by O during the search.  You can calculate this by keeping a list of all board states, or by just counting the number of X's and O's in the final board.

Here are some example results.

In [8]:
def playGame(game,opponent,depthLimit):
    total= -1
    turns= 0
    print(game)
    while not game.isOver():
        if game.player == 'X':
            turns = turns + 1
            total = total + 2
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
            
    return ('Negamax made {} moves. {} moves explored for ebf({}, {}) of {}'.format(turns,game.movesExplored,game.movesExplored,total,ebf(game.movesExplored,total)))

def playGameIDS(game,opponent,depthLimit):
    total= -1
    turns= 0
    print(game)
    while not game.isOver():
        if game.player == 'X':
            turns = turns + 1
            total = total + 2
        score,move = negamaxIDS(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
            
    return ('NegamaxIDS made {} moves. {} moves explored for ebf({}, {}) of {}'.format(turns,game.movesExplored,game.movesExplored,total,ebf(game.movesExplored,total)))

def playGameIDSab(game,opponent,depthLimit):
    total= -1
    turns= 0
    print(game)
    while not game.isOver():
        if game.player == 'X':
            turns = turns + 1
            total = total + 2
        score,move = negamaxIDSab(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
            
    return ('NegamaxIDSab made {} moves. {} moves explored for ebf({}, {}) of {}'.format(turns,game.movesExplored,game.movesExplored,total,ebf(game.movesExplored,total)))

def playGames(opponent,depthLimit):
    print('Negamax')
    game=TTT()
    x= playGame(game,opponent,depthLimit)
    print()
    print('NegamaxIDS')
    game1=TTT()
    y=playGameIDS(game1,opponent,depthLimit)
    print()
    print('NegamaxIDSab')
    game2=TTT()
    z=playGameIDSab(game2,opponent,depthLimit)
    print()
    print()
    print(x)
    print(y)
    print(z)
    return
    


In [9]:
playGames(opponent, 10)

Negamax
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 

NegamaxIDS
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 

NegamaxIDSab
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 


Negama

## Grading

In [10]:
%run -i A4grader.py


Testing negamax starting from ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of 1

--- 10/10 points. negamax correctly explored 124 states.

Testing negamax starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of -1 and move of 5

Testing negamaxIDS with max depth of 5, starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamaxIDS correctly returns value of -1 and move of 5

Testing negamaxIDSab starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 20/20 points. negamaxIDSab correctly returns value of -1 and move of 5

Testing playGame with opponent that always plays in highest numbered position.
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 8
X| | 
-----
 | | 
-----
 | |O
Player X to 2 for score 1
X| |X
-----
 | | 
-----
 | |O
Player O to 7
X| |X
-----
 | | 
-----
 |O|O
Player X to 1 for 

## Extra Credit 

Earn one extra credit point for each of the following.

  - Implement another game and repeat the above steps.

  - Implement a random move chooser as the opponent (Player O) and determine how many times Player X can win against this opponent as an average over multiple games.

#### This section of the code implements a random move chooser as the opponent (Player O) and determine how many times Player X can win against this opponent as an average over multiple games.
#### In this section, I have used NegamaxIDSab function since it is the fastest method. This program evaluates 100 rounds for the Tic-Tac-Toe game

In [11]:
from random import shuffle

In [12]:
def opponentRandom(board, target=' '):
    boardsize = len(board)
    moves = []
    for i in range (boardsize):
        if board[i] == target:
            moves.append(i)
    shuffle(moves)
    return moves.pop() if moves else -1

In [13]:
def playGameNegamaxMultipleRounds(rounds=100):
    
    print("Playing {} games and finding average No of 'X' wins, using NegamaxIDSab Method".format(rounds))
    
    XWins = 0
    value = 0
    
    for round in range(rounds):
        game = TTT()
        print(game)
        while not game.isOver():
            if game.player == 'X':
                value,move = negamaxIDSab(game, 9)
            else:
                move = opponentRandom(game.board)
            if move == None:
                print("move is None. Stopping")
                break
            game.makeMove(move)
            print("Player",game.player,"to",move,"for value",value)
            if not False: print()
            print(game)
            game.changePlayer()
        global isXWon
        if game.Winner == 'X':
            XWins = XWins + 1
        
    print('X won {0} out of {1} games for {2}% wins.'.format(XWins, rounds, (XWins/rounds) * 100) )
    return

In [14]:
playGameNegamaxMultipleRounds()

Playing 100 games and finding average No of 'X' wins, using NegamaxIDSab Method
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for value 1

X| | 
-----
 | | 
-----
 | | 
Player O to 4 for value 1

X| | 
-----
 |O| 
-----
 | | 
Player X to 1 for value 1

X|X| 
-----
 |O| 
-----
 | | 
Player O to 8 for value 1

X|X| 
-----
 |O| 
-----
 | |O
Player X to 2 for value 1

X|X|X
-----
 |O| 
-----
 | |O
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for value 1

X| | 
-----
 | | 
-----
 | | 
Player O to 8 for value 1

X| | 
-----
 | | 
-----
 | |O
Player X to 1 for value 1

X|X| 
-----
 | | 
-----
 | |O
Player O to 2 for value 1

X|X|O
-----
 | | 
-----
 | |O
Player X to 3 for value 1

X|X|O
-----
X| | 
-----
 | |O
Player O to 7 for value 1

X|X|O
-----
X| | 
-----
 |O|O
Player X to 6 for value 1

X|X|O
-----
X| | 
-----
X|O|O
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for value 1

X| | 
-----
 | | 
-----
 | | 
Player O to 8 for value 1

X| | 
-----
 | | 
-----
 | |O
Player X to 1 for value 1

X|X| 

Player X manages to win 96 out of 100 games or with a probablity of 96% chances of winning